In [ ]:
import torch
import math

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    print("CUDA is available. You can use the GPU!")
else:
    print("CUDA is not available. Using the CPU instead.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA is available. You can use the GPU!


# Brownian Motion

In [ ]:
def generate_brownian(n_paths, n_steps):

    dW = torch.randn(n_paths, n_steps)
    dW[:, 0] = 0.0
    W = dW.cumsum(dim=-1)

    return W

In [ ]:
generate_brownian(2,5)

tensor([[ 0.0000, -1.3195, -0.5633, -0.7642,  0.4852],
        [ 0.0000,  0.6495,  1.2836,  2.0777,  0.7174]])

In [ ]:
def generate_geometric_brownian(n_paths=2, n_steps=5, sigma=0.2, dt=1/250):

    t = torch.arange(n_steps) * dt
    W = generate_brownian(n_paths, n_steps)

    return torch.exp((-0.5 * sigma**2) * t + sigma *  torch.sqrt(torch.tensor(dt)) * W)

In [ ]:
generate_geometric_brownian()

tensor([[1.0000, 0.9989, 0.9983, 0.9905, 0.9836],
        [1.0000, 1.0138, 0.9979, 0.9815, 0.9846]])

# Derivative payoff

In [ ]:
def european_payoff(input, call=True, strike=1.0):

    if call:
        return torch.nn.functional.relu(input[..., -1] - strike)
    else:
        return torch.nn.functional.relu(strike - input[..., -1])

In [ ]:
spot = generate_geometric_brownian(2000, 251).unsqueeze(1)
spot

tensor([[[1.0000, 0.9961, 1.0127,  ..., 0.8152, 0.8338, 0.8480]],

        [[1.0000, 0.9846, 0.9994,  ..., 0.9495, 0.9614, 0.9592]],

        [[1.0000, 0.9863, 0.9775,  ..., 1.0236, 1.0177, 1.0289]],

        ...,

        [[1.0000, 1.0248, 1.0190,  ..., 1.0454, 1.0297, 1.0583]],

        [[1.0000, 1.0035, 1.0083,  ..., 1.0685, 1.0715, 1.0458]],

        [[1.0000, 0.9826, 0.9726,  ..., 0.7807, 0.7771, 0.7786]]])

In [ ]:
payoff = european_payoff(spot)
payoff

tensor([[0.0000],
        [0.0000],
        [0.0289],
        ...,
        [0.0583],
        [0.0458],
        [0.0000]])

# profit and loss function (pl)

In [ ]:
def pl(spot, unit, cost=None, payoff=None):

    output = unit[..., :-1].mul(spot.diff(dim=-1)).sum(dim=(-2,-1))

    if payoff is not None:
        output -= payoff.squeeze(-1)

    if cost is not None:
        c = torch.tensor(cost).unsqueeze(0).unsqueeze(-1)
        output -= (spot[..., :-1].mul(unit.diff(dim=-1)).abs() * c).sum(dim=(-2, -1))

    return output

In [ ]:
unit = torch.randn_like(spot)
unit

tensor([[[-1.8705,  1.8441, -1.4028,  ...,  1.3688,  0.0530,  1.9276]],

        [[-0.6972,  0.3534, -0.0903,  ..., -0.8986, -1.9493,  1.9040]],

        [[ 0.5817,  0.4234,  1.6568,  ...,  0.6519, -0.0598,  0.2979]],

        ...,

        [[ 0.0086, -0.1229, -1.0818,  ...,  1.5928,  0.6460, -0.7501]],

        [[ 0.2057,  2.2933, -0.7583,  ..., -0.6832, -0.0532, -1.1174]],

        [[-0.4827, -0.0908, -1.3293,  ..., -1.1238, -0.9680, -1.4733]]])

In [ ]:
pl(spot, unit, cost=0.1)

tensor([-27.7136, -28.0156, -29.8360,  ..., -29.7399, -33.7619, -23.0347])

In [ ]:
pl(spot, unit, cost= 0.1, payoff=payoff)

tensor([-27.7136, -28.0156, -29.8649,  ..., -29.7982, -33.8077, -23.0347])

# Features

In [ ]:
def time_to_maturity(spot, dt):

    n_paths, _, n_steps = spot.size()
    t = torch.arange(n_steps) * dt

    return (t[-1] - t).unsqueeze(0).expand(n_paths, 1, -1)

In [ ]:
time_to_maturity(spot, 0.004)

tensor([[[1.0000, 0.9960, 0.9920,  ..., 0.0080, 0.0040, 0.0000]],

        [[1.0000, 0.9960, 0.9920,  ..., 0.0080, 0.0040, 0.0000]],

        [[1.0000, 0.9960, 0.9920,  ..., 0.0080, 0.0040, 0.0000]],

        ...,

        [[1.0000, 0.9960, 0.9920,  ..., 0.0080, 0.0040, 0.0000]],

        [[1.0000, 0.9960, 0.9920,  ..., 0.0080, 0.0040, 0.0000]],

        [[1.0000, 0.9960, 0.9920,  ..., 0.0080, 0.0040, 0.0000]]])

In [ ]:
def moneyness(spot, strike):
    return spot/strike

In [ ]:
moneyness(spot, 1.1)

tensor([[[0.9091, 0.9056, 0.9207,  ..., 0.7411, 0.7580, 0.7709]],

        [[0.9091, 0.8951, 0.9085,  ..., 0.8632, 0.8740, 0.8720]],

        [[0.9091, 0.8967, 0.8887,  ..., 0.9305, 0.9252, 0.9353]],

        ...,

        [[0.9091, 0.9316, 0.9264,  ..., 0.9503, 0.9361, 0.9621]],

        [[0.9091, 0.9123, 0.9167,  ..., 0.9714, 0.9741, 0.9508]],

        [[0.9091, 0.8933, 0.8842,  ..., 0.7097, 0.7065, 0.7078]]])

In [ ]:
def log_moneyness(spot, strike):
    return torch.log(spot/strike)

In [ ]:
log_moneyness(spot, 1.0)

tensor([[[ 0.0000, -0.0039,  0.0126,  ..., -0.2043, -0.1818, -0.1649]],

        [[ 0.0000, -0.0156, -0.0006,  ..., -0.0519, -0.0393, -0.0416]],

        [[ 0.0000, -0.0138, -0.0227,  ...,  0.0233,  0.0176,  0.0285]],

        ...,

        [[ 0.0000,  0.0245,  0.0188,  ...,  0.0444,  0.0293,  0.0567]],

        [[ 0.0000,  0.0035,  0.0083,  ...,  0.0663,  0.0691,  0.0448]],

        [[ 0.0000, -0.0176, -0.0278,  ..., -0.2476, -0.2521, -0.2503]]])

In [ ]:
def volatility(spot, vol):
    return torch.ones_like(spot) * vol

In [ ]:
volatility(spot, 0.2)

tensor([[[0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000]],

        [[0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000]],

        [[0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000]],

        ...,

        [[0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000]],

        [[0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000]],

        [[0.2000, 0.2000, 0.2000,  ..., 0.2000, 0.2000, 0.2000]]])

# MLP

In [ ]:
from torch import nn

class MLP(nn.Module):

    def __init__(self, n_inputs):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_inputs, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,1)
        )

    def forward(self, x):

        out = self.model(x)

        return out

In [ ]:
m = MLP(3).to(device)

In [ ]:
m

MLP(
  (model): Sequential(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [ ]:
x = torch.randn(3).to(device)
m(x)

tensor([-0.0321], device='cuda:0', grad_fn=<ViewBackward0>)

# Dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
spot.size()

torch.Size([2000, 1, 251])

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.cat(data, dim=1)

    def __len__(self):
        return self.data.size(2)

    def __getitem__(self, index):
        return self.data[:, :, index].unsqueeze(1).to(device)

lm = moneyness(spot, 1.1)
t = time_to_maturity(spot, 0.004)
v = volatility(spot, 0.2)

ds = MyDataset([lm, t, v])

In [ ]:
for i in ds:
    print(i)

tensor([[[0.9091, 1.0000, 0.2000]],

        [[0.9091, 1.0000, 0.2000]],

        [[0.9091, 1.0000, 0.2000]],

        ...,

        [[0.9091, 1.0000, 0.2000]],

        [[0.9091, 1.0000, 0.2000]],

        [[0.9091, 1.0000, 0.2000]]], device='cuda:0')
tensor([[[0.9056, 0.9960, 0.2000]],

        [[0.8951, 0.9960, 0.2000]],

        [[0.8967, 0.9960, 0.2000]],

        ...,

        [[0.9316, 0.9960, 0.2000]],

        [[0.9123, 0.9960, 0.2000]],

        [[0.8933, 0.9960, 0.2000]]], device='cuda:0')
tensor([[[0.9207, 0.9920, 0.2000]],

        [[0.9085, 0.9920, 0.2000]],

        [[0.8887, 0.9920, 0.2000]],

        ...,

        [[0.9264, 0.9920, 0.2000]],

        [[0.9167, 0.9920, 0.2000]],

        [[0.8842, 0.9920, 0.2000]]], device='cuda:0')
tensor([[[0.9174, 0.9880, 0.2000]],

        [[0.9210, 0.9880, 0.2000]],

        [[0.9137, 0.9880, 0.2000]],

        ...,

        [[0.9171, 0.9880, 0.2000]],

        [[0.9107, 0.9880, 0.2000]],

        [[0.8993, 0.9880, 0.2000]]], devic

# compute_hedge

In [ ]:
def compute_hedge(model, ds):
    outputs = []
    for i in ds:
        outputs.append(model(i))

    return torch.cat(outputs, dim=-1)

In [ ]:
compute_hedge(m, ds)

tensor([[[-0.0330, -0.0328, -0.0330,  ..., -0.0114, -0.0113, -0.0113]],

        [[-0.0330, -0.0327, -0.0328,  ..., -0.0111, -0.0111, -0.0111]],

        [[-0.0330, -0.0327, -0.0326,  ..., -0.0110, -0.0110, -0.0109]],

        ...,

        [[-0.0330, -0.0332, -0.0330,  ..., -0.0111, -0.0110, -0.0110]],

        [[-0.0330, -0.0329, -0.0329,  ..., -0.0111, -0.0111, -0.0110]],

        [[-0.0330, -0.0327, -0.0325,  ..., -0.0110, -0.0110, -0.0110]]],
       device='cuda:0', grad_fn=<CatBackward0>)

# compute_portfolio

In [ ]:
def compute_portfolio(model, ds, payoff):

    unit = compute_hedge(model, ds)

    return pl(spot.to(device), unit)

In [ ]:
compute_portfolio(m, ds, None)

tensor([ 0.0030,  0.0006, -0.0016,  ..., -0.0016, -0.0027,  0.0043],
       device='cuda:0', grad_fn=<SumBackward1>)

In [ ]:
def compute_portfolio_2(model, ds, payoff):

    unit = compute_hedge(model, ds)

    return pl(spot.to(device), unit, payoff=payoff.to(device))

In [ ]:
compute_portfolio_2(m, ds, european_payoff(spot))

tensor([ 0.0030,  0.0006, -0.0305,  ..., -0.0599, -0.0485,  0.0043],
       device='cuda:0', grad_fn=<SubBackward0>)

# Optimizer and Training

In [ ]:
optimizer = torch.optim.Adam(m.parameters())

# Loss functions

In [ ]:
def entropic_risk_measure(x):
    return torch.logsumexp(-x, -1) - math.log(x.size(0))

# Training with entropic risk measure

In [ ]:
for i in range(10):
    optimizer.zero_grad()
    cash = compute_portfolio_2(m, ds, european_payoff(spot))
    loss = entropic_risk_measure(cash)
    loss.backward()
    optimizer.step()

    print(loss)

tensor(0.0917, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0837, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0818, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0834, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0827, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0817, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0818, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0821, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0821, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0818, device='cuda:0', grad_fn=<SubBackward0>)


# prev_hedge

In [ ]:
from torch import nn

class MLP(nn.Module):

    def __init__(self, n_inputs):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_inputs, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,1)
        )

    def forward(self, x):

        out = self.model(x)

        return out

In [ ]:
class NewModel(nn.Module):

    def __init__(self, model):
        super().__init__()
        self.model = model
        self.register_buffer("prev_hegde", None)

    def forward(self, x):
        if self.prev_hegde is None:
            self.register_buffer("prev_hedge", torch.zeros(x.size(0), x.size(1), 1).to(device))

        new_x = torch.cat([x, self.prev_hedge], dim=-1)
        out = self.model(new_x)
        self.prev_hedge = out.detach()

        return out

In [ ]:
mm = NewModel(MLP(4)).to(device)
mm

NewModel(
  (model): MLP(
    (model): Sequential(
      (0): Linear(in_features=4, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)

In [ ]:
for i in range(1000):
    optimizer.zero_grad()
    cash = compute_portfolio_2(mm, ds, european_payoff(spot))
    loss = entropic_risk_measure(cash)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(loss)

tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
tensor(0.0919, device='cuda:0', grad_fn=<SubBackward0>)
